<a href="https://colab.research.google.com/github/annawilson123/pseudocode-to-python-sourcecode/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-68wjz_by
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-68wjz_by
  Resolved https://github.com/huggingface/transformers to commit fadb053379b3ef24c4ec8e6d7d58555af21f58db
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.

In [ ]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install auto-gptq

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, GPTQConfig, GenerationConfig
from trl import SFTTrainer
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ananyarn/get_python")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("ananyarn/get_python", low_cpu_mem_usage=True, return_dict=True, torch_dtype=torch.float16, device_map="cuda")

adapter_config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/600M [00:00<?, ?B/s]

In [ ]:
generation_config = GenerationConfig(do_sample=True, top_k=1, temperature=0.1, max_new_tokens=25, pad_token_id=tokenizer.eos_token_id)

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (quant_linear_module): QuantLinear()
           

In [ ]:
input1 = tokenizer("<s>[INST]1. Function radix_sort\n2. Find the largest element\n3. Define the number of passesas the number of digits in the largest element\n4. Iterate by digit position starting from the least significant digit and going up to the most significant digit. \n5. Use a stable sorting algorithm to sort the elements based on the value of the digit in that specific position.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input1, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3

In [ ]:
start = s.find("[INST]")
start += 6

In [ ]:
answer = s[start:end]

In [ ]:
print(answer)

1. Function radix_sort
2. Find the largest element
3. Define the number of passesas the number of digits in the largest element
4. Iterate by digit position starting from the least significant digit and going up to the most significant digit. 
5. Use a stable sorting algorithm to sort the elements based on the value of the digit in that specific position.[/INST] ```python
def radix_sort(arr):
    # Find the largest element
    max_val = max(arr)
    num_digits = len(str(max_val))

    # Sort the array in num_digits passes
    for i in range(num_digits):
        # Sort the array in place
        arr.sort(key=lambda x: x % 10**i)

    return arr


In [ ]:
e = answer.find("[/INST]")
algo = answer[:e]
print(algo)

1. Function radix_sort
2. Find the largest element
3. Define the number of passesas the number of digits in the largest element
4. Iterate by digit position starting from the least significant digit and going up to the most significant digit. 
5. Use a stable sorting algorithm to sort the elements based on the value of the digit in that specific position.


In [ ]:
st = e + 18
code = answer[st:]
print(code)

def radix_sort(arr):
    # Find the largest element
    max_val = max(arr)
    num_digits = len(str(max_val))

    # Sort the array in num_digits passes
    for i in range(num_digits):
        # Sort the array in place
        arr.sort(key=lambda x: x % 10**i)

    return arr


In [ ]:
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Function radix_sort
2. Find the largest element
3. Define the number of passesas the number of digits in the largest element
4. Iterate by digit position starting from the least significant digit and going up to the most significant digit. 
5. Use a stable sorting algorithm to sort the elements based on the value of the digit in that specific position.

####################################################################################################
Code (output):
def radix_sort(arr):
    # Find the largest element
    max_val = max(arr)
    num_digits = len(str(max_val))

    # Sort the array in num_digits passes
    for i in range(num_digits):
        # Sort the array in place
        arr.sort(key=lambda x: x % 10**i)

    return arr


In [ ]:
input2 = tokenizer("<s>[INST]1. AVL Tree Insertion\n2. Function for Standard BST Insertion\n2.1. Start with the root node of the AVL tree.\n2.2. If the tree is empty, the new node becomes the root.\n2.3. Otherwise, compare the key of the new node (`new_key`) with the key of the current node (`current_node`).\n2.3.1. If `new_key` is less than `current_node.key`, move left recursively to the left subtree.\n2.3.2. If `new_key` is greater than `current_node.key`, move right recursively to the right subtree.\n2.4. Once you reach a leaf node (a node with both left and right children as NULL), insert the new node as the child of that leaf node. Update the parent pointers accordingly.\n3. Fuction to Rebalance the Tree (Starting from the Newly Inserted Node)\n3.1. Set the current node to the newly inserted node.\n3.2. While the current node is not NULL:\n3.2.1. Calculate the heights of the left and right subtrees of the current node.\n3.2.2. Calculate the balance factor of the current node (left subtree height - right subtree height).\n3.2.3. Check for Imbalance\n3.2.3.1. If the balance factor is greater than 1, the node is unbalanced on the left side. Proceed to step 3 for Left-Side Imbalance.\n3.2.3.2. If the balance factor is less than -1, the node is unbalanced on the right side. Proceed to step 4 for Right-Side Imbalance.\n3.2.4.Set the current node to its parent node to continue traversing upwards.\n4. Left-Side Imbalance\n4.1. Left-Left Case: The new node is on the left side of its child (most common case). Perform a Right Rotation around the current unbalanced node.\n4.2. Left-Right Case: The new node is on the right side of its child. Perform a Left Rotation on the child node, followed by a Right Rotation on the current unbalanced node. \n5. Right-Side Imbalance\n5.1. Right-Right Case: The new node is on the right side of its child (most common case). Perform a Left Rotation around the current unbalanced node.\n5.2. Right-Left Case: The new node is on the left side of its child. Perform a Right Rotation on the child node, followed by a Left Rotation on the current unbalanced node.\n6. Continue Traversal and Rotations: After performing a rotation, repeat step 3.2  for the new current node (which is the node resulting from the rotation). This ensures the balance factors are recalculated and further rotations are performed if needed as we traverse back up the tree.\n7. The rebalancing process stops when either a balanced node is encountered while traversing upwards (balance factor within -1 and +1) or the root of the tree is reached (implying the tree was already balanced).[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input2, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. AVL Tree Insertion
2. Function for Standard BST Insertion
2.1. Start with the root node of the AVL tree.
2.2. If the tree is empty, the new node becomes the root.
2.3. Otherwise, compare the key of the new node (`new_key`) with the key of the current node (`current_node`).
2.3.1. If `new_key` is less than `current_node.key`, move left recursively to the left subtree.
2.3.2. If `new_key` is greater than `current_node.key`, move right recursively to the right subtree.
2.4. Once you reach a leaf node (a node with both left and right children as NULL), insert the new node as the child of that leaf node. Update the parent pointers accordingly.
3. Fuction to Rebalance the Tree (Starting from the Newly Inserted Node)
3.1. Set the current node to the newly inserted node.
3.2. While the current node is not NULL:
3.2.1. Calculate the heights of the left and right subtrees of

In [ ]:
print(s)

[INST]1. AVL Tree Insertion
2. Function for Standard BST Insertion
2.1. Start with the root node of the AVL tree.
2.2. If the tree is empty, the new node becomes the root.
2.3. Otherwise, compare the key of the new node (`new_key`) with the key of the current node (`current_node`).
2.3.1. If `new_key` is less than `current_node.key`, move left recursively to the left subtree.
2.3.2. If `new_key` is greater than `current_node.key`, move right recursively to the right subtree.
2.4. Once you reach a leaf node (a node with both left and right children as NULL), insert the new node as the child of that leaf node. Update the parent pointers accordingly.
3. Fuction to Rebalance the Tree (Starting from the Newly Inserted Node)
3.1. Set the current node to the newly inserted node.
3.2. While the current node is not NULL:
3.2.1. Calculate the heights of the left and right subtrees of the current node.
3.2.2. Calculate the balance factor of the current node (left subtree height - right subtree he

In [ ]:
input3 = tokenizer("<s>[INST]1. Function Depth-First Search (DFS) This is a recursive function that takes the vertex (vertex) as input.\n2. Mark the current vertex as visited (visited[vertex] = True).\n3. Iterate through all adjacent vertices (neighbor) of the current vertex:\n4. If the neighbor is not visited (visited[neighbor] == False):\n4.1. In directed graphs, update the parent information (parent[neighbor] = vertex).\n4.2 Recursively call DFS on the neighbor (DFS(neighbor))\n5. Function Detect_Cycle\n6. Create a visited array (visited) to track visited nodes. Initialize all elements in visited to False.\n7. Create a parent array (parent) to track the parent node during DFS traversal (used for directed graphs). Initialize all elements in parent to None.\n8. Iterate over Vertices. Loop through all the vertices in the graph. If a vertex is not visited yet (visited[vertex] == False), perform a DFS traversal starting from that vertex.\n9. If the neighbor is already visited and it's not the parent of the current vertex in directed graphs (visited[neighbor] == True and (parent is None or parent[neighbor] != vertex) in directed graphs) A cycle is detected! There's a back edge to a previously visited node (excluding the parent in directed graphs).\n10. You can either return True (to stop further DFS) or raise an exception indicating a cycle is found.\n11. If the DFS completes iterating through all vertices without detecting a cycle then the graph does not contain a cycle.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input3, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Function Depth-First Search (DFS) This is a recursive function that takes the vertex (vertex) as input.
2. Mark the current vertex as visited (visited[vertex] = True).
3. Iterate through all adjacent vertices (neighbor) of the current vertex:
4. If the neighbor is not visited (visited[neighbor] == False):
4.1. In directed graphs, update the parent information (parent[neighbor] = vertex).
4.2 Recursively call DFS on the neighbor (DFS(neighbor))
5. Function Detect_Cycle
6. Create a visited array (visited) to track visited nodes. Initialize all elements in visited to False.
7. Create a parent array (parent) to track the parent node during DFS traversal (used for directed graphs). Initialize all elements in parent to None.
8. Iterate over Vertices. Loop through all the vertices in the graph. If a vertex is not visited yet (visited[vertex] == False), perform a DFS trave

In [ ]:
input3 = tokenizer("<s>[INST]1. Define a distance array (`dist[]`) to store the shortest distances from the source vertex to all other vertices. Initialize all elements in `dist[]` to infinity (except the source vertex).\n2. Define a predecessor array (`pred[]`) to store the predecessor node in the shortest path (optional for path reconstruction). Initialize all elements in `pred[]` to None (or any default value).\n3. Choose a source vertex (`source`).\n4. Iterate for |V| - 1 times (where |V| is the number of vertices in the graph):\n4.1. For each edge (`u`, `v`, `weight`) in the graph:\n4.2. Relax the edge: Calculate the tentative distance from the source to `v` through `u` (`new_dist = dist[u] + weight`).\n4.3. Update the distance array (`dist[v]`) and predecessor array (`pred[v]`) if the tentative distance is shorter than the current distance.\n4.3.1. If `new_dist` is less than `dist[v]`:\n4.3.2. Update `dist[v] = new_dist`.\n4.3.3. Update `pred[v] = u` (to store the predecessor for path reconstruction).\n5 Perform one additional relaxation iteration (|V|th iteration) over all edges.\n5.1. If any distance is updated in this extra iteration, it indicates a negative-weight cycle exists in the graph.\n5.2. You can either stop the algorithm and report a negative-weight cycle or modify it to handle such cycles (more complex).\n6. The `dist[]` array now contains the shortest distances from the source vertex to all other vertices (if no negative-weight cycles exist).\n7. The `pred[]` array (optional) can be used to reconstruct the shortest paths by backtracking from the destination vertex to the source using the predecessor information.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input3, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Define a distance array (`dist[]`) to store the shortest distances from the source vertex to all other vertices. Initialize all elements in `dist[]` to infinity (except the source vertex).
2. Define a predecessor array (`pred[]`) to store the predecessor node in the shortest path (optional for path reconstruction). Initialize all elements in `pred[]` to None (or any default value).
3. Choose a source vertex (`source`).
4. Iterate for |V| - 1 times (where |V| is the number of vertices in the graph):
4.1. For each edge (`u`, `v`, `weight`) in the graph:
4.2. Relax the edge: Calculate the tentative distance from the source to `v` through `u` (`new_dist = dist[u] + weight`).
4.3. Update the distance array (`dist[v]`) and predecessor array (`pred[v]`) if the tentative distance is shorter than the current distance.
4.3.1. If `new_dist` is less than `dist[v]`:
4.3.2. Upda

In [ ]:
input5 = tokenizer("<s>[INST]1. Define a node structure with two fields, data: This field stores the data element of the stack, next: This field stores a reference (pointer) to the next node in the linked list.\n2. Maintain a variable top which points to the topmost element of the stack. Initially, top is set to NULL (or None in some languages) indicating an empty stack.\n3. Write a function push(data). This function inserts a new element (data) onto the top of the stack.\n3.1. Create a new node (newNode).\n3.2. Assign the data to the newNode.data field.\n3.3. Set the newNode.next to point to the current top node.\n3.4. Update the top to point to the newNode.\n4. Write a function pop(). This function removes and returns the element from the top of the stack.\n4.1. Check if the stack is empty (top == NULL). If yes, return an error (or None).\n4.2. Store the data of the top node (top.data).\n4.3. Update the top to point to the next node (top = top.next).\n4.4. Return the stored data (data).\n5. Write a function peek(): This function returns the data element at the top of the stack without removing it.\n5.1. Check if the stack is empty (top == NULL). If yes, return an error (or None).\n5.2. Return the data of the top node (top.data).\n6. Write a function isEmpty(): This function checks if the stack is empty.\n6.1. Return True if top is NULL, False otherwise.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input5, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Define a node structure with two fields, data: This field stores the data element of the stack, next: This field stores a reference (pointer) to the next node in the linked list.
2. Maintain a variable top which points to the topmost element of the stack. Initially, top is set to NULL (or None in some languages) indicating an empty stack.
3. Write a function push(data). This function inserts a new element (data) onto the top of the stack.
3.1. Create a new node (newNode).
3.2. Assign the data to the newNode.data field.
3.3. Set the newNode.next to point to the current top node.
3.4. Update the top to point to the newNode.
4. Write a function pop(). This function removes and returns the element from the top of the stack.
4.1. Check if the stack is empty (top == NULL). If yes, return an error (or None).
4.2. Store the data of the top node (top.data).
4.3. Update the 

In [ ]:
input6 = tokenizer("<s>[INST]1. Find the largest element in the `array`. Let's call this value `max`.\n2. Create a Count Array\n2.1. Create a new array called `count_array` with a size of `max + 1`.\n2.2. Initialize all elements in `count_array` to 0. This array will store the frequency (count) of each unique element in the original array.\n3. Count Element Occurrences\n3.1. Iterate through the original `array`\n3.2. For each element `array[i]`, find its index `i` in the `count_array`.\n3.3. Increment the value at that index (`count_array[i]`) by 1. This increases the count of the specific element encountered in the original array.\n4. Calculate Cumulative Sums\n4.1. Iterate through the `count_array` from index 1 to `max`:\n4.2. For each index `i` (starting from 1), add the value at the previous index (`count_array[i-1]`) to the current value (`count_array[i]`). This creates a cumulative sum of element frequencies.\n5. Restore Elements (Stable Sorting)\n5.1. Create a new empty array (`sorted_array`) of the same size `n` as the original array. This will store the sorted elements.\n5.2. Iterate through the original `array` **in reverse order** (from `n-1` to 0):\n5.2.1. Get the value of the current element `array[i]`.\n5.2.2. Find the corresponding count `count_array[array[i]]` in the `count_array`. This value represents the position where this element should be placed in the sorted array.\n5.2.3. Decrement the count (`count_array[array[i]]`) by 1 as you are placing one element of that value.\n5.2.4. Place the current element `array[i]` at the index `count_array[array[i]]` in the `sorted_array`.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input6, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Find the largest element in the `array`. Let's call this value `max`.
2. Create a Count Array
2.1. Create a new array called `count_array` with a size of `max + 1`.
2.2. Initialize all elements in `count_array` to 0. This array will store the frequency (count) of each unique element in the original array.
3. Count Element Occurrences
3.1. Iterate through the original `array`
3.2. For each element `array[i]`, find its index `i` in the `count_array`.
3.3. Increment the value at that index (`count_array[i]`) by 1. This increases the count of the specific element encountered in the original array.
4. Calculate Cumulative Sums
4.1. Iterate through the `count_array` from index 1 to `max`:
4.2. For each index `i` (starting from 1), add the value at the previous index (`count_array[i-1]`) to the current value (`count_array[i]`). This creates a cumulative sum of element fre

In [ ]:
input7 = tokenizer("<s>[INST]1. Define an array queue to store the elements of the queue.\n2. Maintain two variables front and rear to track the indices of the front and rear elements in the queue, respectively. front: Initially points to -1 (indicating an empty queue). rear: Initially points to -1 (indicating an empty queue).\n3.Create a function enqueue(data): This function inserts a new element (data) into the rear of the queue.\n3.1. Check if the queue is full:\n3.1.1. If yes, print an error message (Queue Overflow).\n3.2. Otherwise: \n3.2.1. If the queue is empty, set both front and rear to 0.\n3.2.2. Otherwise, increment rear by 1 and perform the modulo operation with the queue size.\n3.2.3. Assign the data to the queue[rear] position.\n4. Create a function dequeue(): This function removes and returns the element from the front of the queue.\n4.1. Check if the queue is empty (front == -1):\n4.1.1. If yes, print an error message (Queue Underflow).\n4.1.2. Return None (or an appropriate indicator).\n4.2. Otherwise:\n4.2.1. Store the element at the front (temp = queue[front]).\n4.2.2. If there's only one element (front == rear), set both front and rear to -1 (empty queue).\n4.2.3. Otherwise, increment front by 1 and perform the modulo operation with the queue size (front = (front + 1) % size).\n4.3.4. Return the stored element (temp).\n5. Create a function printQueue(): This function prints the contents of the queue.\n5.1. If the queue is empty (front == -1):\n5.1.1. Print Queue is empty.\n5.2. Otherwise:\n5.2.1. Use a loop to iterate from front to rear (considering circular nature):\n5.2.1.1. Print the element at queue[i].\n5.2.1.2. If i is not rear, print a delimiter to separate elements.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input7, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Define an array queue to store the elements of the queue.
2. Maintain two variables front and rear to track the indices of the front and rear elements in the queue, respectively. front: Initially points to -1 (indicating an empty queue). rear: Initially points to -1 (indicating an empty queue).
3.Create a function enqueue(data): This function inserts a new element (data) into the rear of the queue.
3.1. Check if the queue is full:
3.1.1. If yes, print an error message (Queue Overflow).
3.2. Otherwise: 
3.2.1. If the queue is empty, set both front and rear to 0.
3.2.2. Otherwise, increment rear by 1 and perform the modulo operation with the queue size.
3.2.3. Assign the data to the queue[rear] position.
4. Create a function dequeue(): This function removes and returns the element from the front of the queue.
4.1. Check if the queue is empty (front == -1):
4.1.1. If 

In [ ]:
input8 = tokenizer("<s>[INST]1. Define a node structure with three fields: data: This field stores the data element of the node, next: This field stores a reference (pointer) to the next node in the list, prev: This field stores a reference (pointer) to the previous node in the list.\n2. Maintain two variables: head: This points to the first node in the list (or None if empty), tail: This points to the last node in the list (or None if empty).\n3. Write a function for insertion. \n3.1. Insert at the beginning (head)\n3.1.1. Create a new node with the data.\n3.1.2. Set the next of the new node to the current head.\n3.1.3. If the list is not empty, update the prev of the current head to point to the new node.\n3.1.4. Update the head to point to the new node.\n3.2. Insert at the end (tail)\n3.2.1. Create a new node with the data.\n3.2.2. If the list is empty, set both head and tail to the new node.\n3.2.3. Otherwise, set the next of the current tail to the new node.\n3.2.4. Update the prev of the new node to point to the current tail.\n3.2.5. Update the tail to point to the new node.\n4. Write a function for Deletion\n4.1. Delete at the beginning (head)\n4.1.1. If the list is empty, do nothing (or handle error).\n4.1.2. Store the data of the current head node.\n4.1.3. Update the head to point to the next node.\n4.1.4. If the new head is not None, update its prev to be None.\n4.1.5. Return the stored data (of the deleted node).\n4.2. Delete at the end (tail)\n4.2.1. If the list is empty, do nothing (or handle error).\n4.2.2. Store the data of the current tail node.\n4.2.3. Update the tail to point to the prev node.\n4.2.4. If the new tail is not None, update its next to be None.\n4.3.5. Return the stored data (of the deleted node).\n5. Write a function for Search. You can traverse the list starting from the head and compare the data of each node with the target value. If a match is found, return the node or its data.\n6. Write a function for Traversal. You can traverse the list forward or backward using the next and prev pointers of the nodes, respectively.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input8, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Define a node structure with three fields: data: This field stores the data element of the node, next: This field stores a reference (pointer) to the next node in the list, prev: This field stores a reference (pointer) to the previous node in the list.
2. Maintain two variables: head: This points to the first node in the list (or None if empty), tail: This points to the last node in the list (or None if empty).
3. Write a function for insertion. 
3.1. Insert at the beginning (head)
3.1.1. Create a new node with the data.
3.1.2. Set the next of the new node to the current head.
3.1.3. If the list is not empty, update the prev of the current head to point to the new node.
3.1.4. Update the head to point to the new node.
3.2. Insert at the end (tail)
3.2.1. Create a new node with the data.
3.2.2. If the list is empty, set both head and tail to the new node.
3.2.3. Oth

In [ ]:
input9 = tokenizer("<s>[INST]1. Define an array queue to store the elements of the queue.\n2.Maintain two variables front and rear to track the indices of the front and rear elements in the queue, respectively. front: Initially points to -1 (indicating an empty queue). rear: Initially points to -1 (indicating an empty queue).\n3. Function enqueue(data): This function inserts a new element (data) into the rear of the queue.\n3.1. Check if the queue is full (rear == size - 1 where size is the size of the array):\n3.2. If yes, print an error message (Queue Overflow).\n3.3. Otherwise:\n3.3.1. If the queue is empty (front == -1), set both front and rear to 0.\n3.3.2. Otherwise, increment rear by 1.\n3.3.3. Assign the data to the queue[rear] position.\n4. Function dequeue(): This function removes and returns the element from the front of the queue.\n4.1. Check if the queue is empty (front == -1):\n4.2. If yes, print an error message (Queue Underflow).\n4.3. Return None (or an appropriate indicator).\n4.4. Otherwise:\n4.4.1. Store the element at the front (temp = queue[front]).\n4.4.2.Increment front by 1.\n4.4.3.If the queue becomes empty after dequeue (front == size), reset both front and rear to -1.\n4.4.4. Return the stored element (temp).\n5. Function isEmpty(): This function checks if the queue is empty.\n5.1. Return True if front == -1, False otherwise.\n6. Function isFull(): This function checks if the queue is full.\n6.1.Return True if rear == size - 1, False otherwise. (Consider the array size)[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input9, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Define an array queue to store the elements of the queue.
2.Maintain two variables front and rear to track the indices of the front and rear elements in the queue, respectively. front: Initially points to -1 (indicating an empty queue). rear: Initially points to -1 (indicating an empty queue).
3. Function enqueue(data): This function inserts a new element (data) into the rear of the queue.
3.1. Check if the queue is full (rear == size - 1 where size is the size of the array):
3.2. If yes, print an error message (Queue Overflow).
3.3. Otherwise:
3.3.1. If the queue is empty (front == -1), set both front and rear to 0.
3.3.2. Otherwise, increment rear by 1.
3.3.3. Assign the data to the queue[rear] position.
4. Function dequeue(): This function removes and returns the element from the front of the queue.
4.1. Check if the queue is empty (front == -1):
4.2. If yes, pr

In [ ]:
input10 = tokenizer("<s>[INST]1. Initialize low and high indices to the start and end of the array, respectively.\n2. Calculate the probe position using the interpolation formula - Interpolation Formula = low + [(high - low) * (X - A[low])]/(A[high] - A[low])] where low - Left pointer, high - Right pointer, A[low] - Element at the left pointer, A[high] - Element at the right pointer, X - Target element to be searched\n3. Compare the probe element with the target element.\n3.1. If they are equal, the search is successful.\n3.2. If the probe element is greater, update the high index to the probe position minus one.\n3.3. If the probe element is smaller, update the low index to the probe position plus one.\n4. Repeat steps 2-3 until the target element is found or the low index exceeds the high index.[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input10, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3
start = s.find("[INST]")
start += 6
answer = s[start:end]
e = answer.find("[/INST]")
algo = answer[:e]
st = e + 18
code = answer[st:]
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1. Initialize low and high indices to the start and end of the array, respectively.
2. Calculate the probe position using the interpolation formula - Interpolation Formula = low + [(high - low) * (X - A[low])]/(A[high] - A[low])] where low - Left pointer, high - Right pointer, A[low] - Element at the left pointer, A[high] - Element at the right pointer, X - Target element to be searched
3. Compare the probe element with the target element.
3.1. If they are equal, the search is successful.
3.2. If the probe element is greater, update the high index to the probe position minus one.
3.3. If the probe element is smaller, update the low index to the probe position plus one.
4. Repeat steps 2-3 until the target element is found or the low index exceeds the high index.

####################################################################################################
Code 